<a href="https://colab.research.google.com/github/versant2612/jnotebooks/blob/main/Document_Similarities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [143]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from textblob import Word
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from unicodedata import normalize


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [144]:
# Sample corpus
documents = [
"Tornar a PUC-Rio referência nacional e internacional em meio ambiente, contribuindo através da ciência e da educação para o desenvolvimento sustentável\
, visando estabelecer a interação entre a Universidade e o meio, e entre sociedade e natureza. A Pontifícia Universidade Católica do Rio de Janeiro é \
uma instituição de direito privado sem fins lucrativos que prima pela produção e transmissão do saber, baseando-se no respeito aos valores humanos e na \
ética cristã, visando acima de tudo o benefício da sociedade. A PUC-Rio busca a excelência na pesquisa, no ensino e na extensão para a formação de \
profissionais competentes, habilitados ao pleno desempenho de suas funções. Esses profissionais são inseridos na realidade brasileira e formados para \
colocar a ciência e a técnica sempre a serviço do homem, colaborando através dos conhecimentos adquiridos na Universidade para a construção de um mundo \
melhor, de acordo com as exigências da justiça do amor cristão. O Núcleo Interdisciplinar de Meio Ambiente – NIMA, teve a sua fundação no ano de 1999, \
pelo atual Reitor da PUC-Rio, Pe. Josafá Carlos de Siqueira S.J. Com o objetivo de ser o local de discussões interdisciplinares sobre as questões \
socioambientais. Além de aglutinar competências internas, o NIMA também se propõe a estabelecer interação entre a Pontifícia Universidade Católica e o \
meio, como cabe às Unidades Complementares de acordo com o Estatuto da Universidade. Há doze anos, o NIMA vem realizando projetos em parceria com \
escolas, empresas, municípios e instituições nacionais e internacionais. O compromisso assumido desde a sua fundação é com a ética ambiental, e assim \
atua para a transformação da cultura antropocêntrica, acreditando na possibilidade de criar novos cenários a partir da comunhão do ser humano com o \
ambiente. A PUC-Rio criou em 2009 a Agenda Ambiental, documento inédito em uma universidade brasileira que reúne a visão de sustentabilidade de um \
grupo multidisciplinar de professores e alunos. Neste documento estão as diretrizes e metas, a curto, médio e longo prazo, para a sustentabilidade na \
Universidade, tanto a nível de gestão quanto de ensino e pesquisa. A iniciativa pioneira surgiu com a “Comissão de Sustentabilidade” criada depois da \
participação da PUC-Rio no Colóquio Global de Reitores que aconteceu em Nova York, por iniciativa do Secretário Geral das Nações Unidas Ban Ki-Moon. \
O plantio de 15 árvores no estacionamento do campus simbolizou o início das ações. A coordenação do projeto é feita pelo NIMA, e entre as propostas \
estão a troca de asfalto das ruas do estacionamento para facilitar a drenagem natural de água; uma tecnologia de filtragem que permite a reutilização \
dos recursos hídricos; a implantação de programas que integrem as fontes naturais de energia às alternativas como eólica e solar; o monitoramento \
detalhado dos gases produzidos nos laboratórios da Universidade; e a criação de um centro de reciclagem."
,
"Maria Celina Bodin de Moraes (Doutorado em direito civil, Università di Camerino, 1986) é Professora Titular de Direito Civil da Faculdade de \
Direito da Universidade do Estado do Rio de Janeiro (UERJ) e Professora Associada do Departamento de Direito da Pontifícia Universidade Católica do \
Rio de Janeiro (PUC-Rio). É ábitra e parecerista em Direito Privado e Editora-chefe da Civilistica.com - Revista Eletrônica de direito civil \
(Qualis A2). Foi diretora do Departamento de Direito da PUC-Rio e coordenadora da área do Direito da Faperj. Atua na área do direito privado, \
principalmente mediante a metodologia do direito civil constitucional. Os temas mais frequentes de sua produção científica são: princípios \
constitucionais da dignidade da pessoa humana e da solidariedade, aplicação direta da Constituição às relações privadas, proteção das pessoas \
vulneráveis, direitos da personalidade, relações familiares, contratos e responsabilidade civil."       
,
"Possui graduação em Engenharia Mecânica pela Pontifícia Universidade Católica do Rio de Janeiro (1982), mestrado em Engenharia de Produção - \
Departamento de Engenharia Industrial (1996) e doutorado em Engenharia de Produção pela Pontifícia Universidade Católica do Rio de Janeiro (2005). \
O mestrado e doutorado foram ambos na área de Finanças e Análise de Investimentos. Foi Consultor Master da Controladoria - Petróleo Brasileiro - \
Rio de Janeiro - Matriz, onde trabalhou de 01/02/1983 até 05/12/2016. É Professor Adjunto do Departamento de Engenharia Elétrica - PUC-Rio, sendo \
pesquisador do Grupo de Sistemas de Apoio à Decisão - Laboratório de Inteligência Computacional Aplicada. É autor de dois livros texto (pós-graduação) \
de opções reais. É Corporate Advisor do Real Options Group. Ministrou cursos de opções reais na Petrobras de 1996 a 2016, além de cursos de teoria \
dos jogos e valor da informação até 2016. Lá ministrou outros cursos como de análise de risco em projetos, finanças corporativas e project finance. \
Na PUC-Rio ministra curso de opções reais desde 2005 e de jogos de opções reais desde 2006. Coordenou vários projetos de pesquisa entre a Petrobras \
e universidades (principalmente PUC-Rio, mas também FGV e Mines Paris Tech). Já foi orador principal em várias conferências internacionais. Já deu \
seminários de opções reais em instituições internacionais (como o MIT em 2002 e 2003) e entidades nacionais (como a Unicamp em 1998, UFRJ em 2003 e \
a Alliance e Fundação Dom Cabral em 2005). Criou o primeiro website dedicado a opções reais (na PUC em 1995). Desenvolveu várias aplicações \
(inclusive software) de opções reais na Petrobras, onde também implantou padrões de análise de risco, de análise de investimento em informação e \
outros. Tem experiência na área de Engenharia de Produção, com ênfase em Avaliação Econômica de Projetos, atuando principalmente nos seguintes temas: \
opções reais, análise de investimentos sob incertezas, jogos de opções reais, teoria dos jogos, exploração e produção de petróleo, valor da informação, \
análise de risco em projetos, análise de investimento em energia renovável, custo de capital de projetos e financiamento corporativo."
,
"Doutora em Serviço Social (UERJ-2015). Mestre em Serviço Social (UFRJ -2010). Graduada em Serviço Social (UFRJ -1999). Possui experiência em \
diversas áreas do trabalho profissional do Serviço Social ( Empresa, AssistÊncia Social, Saúde, consultoria/assessoria). É docente e pesquisadora da \
Pontifícia Universidade Católica do Rio de Janeiro nos cursos de graduação e pós-graduação em Serviço Social, trabalhou em empresas multinacionais \
(Farmacêutica - GlaxoSmithKline e Telefonia celular- Claro), ONG (Obra Social do Rio de Janeiro), no setor público atuou na Prefeitura do Rio de \
Janeiro na condição de concursada. Autora do Livro Serviço Social e Consultoria Empresarial (2020), organizadora de coletâneas sobre trabalho, \
Serviço Social e Direitos Humanos."
,
"Engenheiro Mecânico formado pela Universidade Federal do Rio de Janeiro (1980), MSc em Engenharia Mecânica pela Pontifícia Universidade Católica \
do Rio de Janeiro (1988) e PhD em Engenharia Mecânica - Tecnologia de Aerossóis pela University of Minnesota (1994), USA. Professor Associado do \
Departamento de Engenharia Mecânica da Pontifícia Universidade Católica do Rio de Janeiro, onde coordena o grupo de pesquisas multidisciplinar de \
Engenharia Ambiental - Poluição Atmosférica atuando nos seguintes temas: tecnologia de aerossóis e instrumentação, processos atmosféricos, \
biocombustíveis, controle da poluição (poluentes locais e gases de efeito estufa), captura e armazenamento geológico de carbono, desenvolvimento de \
sensores ambientais, e geração de energia através de resíduos sólidos urbanos (Waste to Energy, WTE). Engenheiro de Petróleo pela Petrobras (CEPET 81), \
onde atuou entre 1981-1985. Membro do IPCC - Intergovernmental Panel on Climate Change (entre 2004-2017), grupo de cientistas que recebeu o Prêmio Nobel\
 da Paz em 2007."
,
"Flávia de Almeida Viveiros de Castro é Juíza de Direito e Professora Convidada da EMERJ (Escola da Magistratura do Estado do Rio de Janeiro) ESAJ \
(Escola de Administração Judiciária do Tribunal de Justiça do Estado do Rio de Janeiro) e Fundação Getúlio Vargas - RJ; Possui Pós-Doutorado em \
Direitos Humanos pelo Instituto JUS GENTIUM da Faculdade de Direito de Coimbra (2017/18). É doutora em Direito o Civil-Constitucional pela Universidade \
de Direito do Estado do Rio de Janeiro(2006). Mestre em Direito pela Pontifícia Universidade Católica do Rio de Janeiro (1999); Mestre em Ciência \
Política (Ciência Política e Sociologia) pela Pontifícia Universidade Católica do Rio de Janeiro (1990); Possui curso de Especialização em direito \
português pelo Centro de Estudos Judiciários de Portugal (1997) e Especialização em Ciência Política, pelo Instituto Metodista Bennett (1985); Graduada \
em Direito e em Comunicação Social, especialidade Jornalismo pela Pontifícia Universidade Católica do Rio de Janeiro (1980 e 1990). É Juíza Titular da \
6ª Vara Cível Regional da Barra da Tijuca, tendo exercido a magistratura em varas de falência, fazenda pública, criminal, tribunal do júri, orfãos e \
sucessões e família. Já assumiu a função de Juíza Eleitoral na 175 Zona Eleitoral. Já exerceu a mesma função em Marechal Hermes, Bangu e Pavuna. Foi \
Juíza Corregedora - Auxiliar da Corregedora de Justiça em (1996);Exerce a docência como palestrante e professora convidada nas áreas de Direito Civil \
e Direito Tributário. Foi coordenadora acadêmica do Instituto de direito da Pontifícia Universidade Católica do Rio de Janeiro e coordenadora acadêmica \
dos cursos de pós-graduação em direito imobiliário, direito fiscal, direito empresarial, direito processual civil e direito da saúde do Instituto de \
Direito da PUC-RJ."
,
"Vitor Nascimento Aguiar graduou-se em Engenharia Civil com ênfase em Mecânica dos Solos na Universidade Federal do Rio de Janeiro (UFRJ) em 2006. \
Concluiu o mestrado e o doutorado em Engenharia Civil (Geotecnia) pela COPPE/UFRJ em 2008 e 2014. Em suas pesquisas de mestrado e doutorado, investigou \
efeitos de tempo e de velocidade de deformação no adensamento e na resistência ao cisalhamento de argilas moles saturadas. Atuou por dez anos em \
projetos e consultorias de obras de infraestrutura na Mecasolo Engenharia e Consultoria. Realizou estágio de pós-doutorado no Norwegian Geotechnical \
Institute (NGI) de 2017 a 2018. Atualmente é professor do quadro principal do Departamento de Engenharia Civil e Ambiental da Pontifícia Universidade \
Católica do Rio de Janeiro (PUC-Rio), onde leciona a disciplina de Laboratório de Geotecnia na graduação e as disciplinas de Ensaios de Laboratório e \
Geotecnia Experimental na pós-graduação. Tem exercido o cargo de secretário executivo na Associação Brasileira de Mecânica dos Solos - Núcleo RJ."
]

In [145]:
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', 0)

In [146]:
documents_df=pd.DataFrame(documents,columns=['documents'])

### Corpus Dataframe

In [147]:
documents_df

,documents
0,"Tornar a PUC-Rio referência nacional e internacional em meio ambiente, contribuindo através da ciência e da educação para o desenvolvimento sustentável, visando estabelecer a interação entre a Universidade e o meio, e entre sociedade e natureza. A Pontifícia Universidade Católica do Rio de Janeiro é uma instituição de direito privado sem fins lucrativos que prima pela produção e transmissão do saber, baseando-se no respeito aos valores humanos e na ética cristã, visando acima de tudo o benefício da sociedade. A PUC-Rio busca a excelência na pesquisa, no ensino e na extensão para a formação de profissionais competentes, habilitados ao pleno desempenho de suas funções. Esses profissionais são inseridos na realidade brasileira e formados para colocar a ciência e a técnica sempre a serviço do homem, colaborando através dos conhecimentos adquiridos na Universidade para a construção de um mundo melhor, de acordo com as exigências da justiça do amor cristão. O Núcleo Interdisciplinar de Meio Ambiente – NIMA, teve a sua fundação no ano de 1999, pelo atual Reitor da PUC-Rio, Pe. Josafá Carlos de Siqueira S.J. Com o objetivo de ser o local de discussões interdisciplinares sobre as questões socioambientais. Além de aglutinar competências internas, o NIMA também se propõe a estabelecer interação entre a Pontifícia Universidade Católica e o meio, como cabe às Unidades Complementares de acordo com o Estatuto da Universidade. Há doze anos, o NIMA vem realizando projetos em parceria com escolas, empresas, municípios e instituições nacionais e internacionais. O compromisso assumido desde a sua fundação é com a ética ambiental, e assim atua para a transformação da cultura antropocêntrica, acreditando na possibilidade de criar novos cenários a partir da comunhão do ser humano com o ambiente. A PUC-Rio criou em 2009 a Agenda Ambiental, documento inédito em uma universidade brasileira que reúne a visão de sustentabilidade de um grupo multidisciplinar de professores e alunos. Neste documento estão as diretrizes e metas, a curto, médio e longo prazo, para a sustentabilidade na Universidade, tanto a nível de gestão quanto de ensino e pesquisa. A iniciativa pioneira surgiu com a “Comissão de Sustentabilidade” criada depois da participação da PUC-Rio no Colóquio Global de Reitores que aconteceu em Nova York, por iniciativa do Secretário Geral das Nações Unidas Ban Ki-Moon. O plantio de 15 árvores no estacionamento do campus simbolizou o início das ações. A coordenação do projeto é feita pelo NIMA, e entre as propostas estão a troca de asfalto das ruas do estacionamento para facilitar a drenagem natural de água; uma tecnologia de filtragem que permite a reutilização dos recursos hídricos; a implantação de programas que integrem as fontes naturais de energia às alternativas como eólica e solar; o monitoramento detalhado dos gases produzidos nos laboratórios da Universidade; e a criação de um centro de reciclagem."
1,"Maria Celina Bodin de Moraes (Doutorado em direito civil, Università di Camerino, 1986) é Professora Titular de Direito Civil da Faculdade de Direito da Universidade do Estado do Rio de Janeiro (UERJ) e Professora Associada do Departamento de Direito da Pontifícia Universidade Católica do Rio de Janeiro (PUC-Rio). É ábitra e parecerista em Direito Privado e Editora-chefe da Civilistica.com - Revista Eletrônica de direito civil (Qualis A2). Foi diretora do Departamento de Direito da PUC-Rio e coordenadora da área do Direito da Faperj. Atua na área do direito privado, principalmente mediante a metodologia do direito civil constitucional. Os temas mais frequentes de sua produção científica são: princípios constitucionais da dignidade da pessoa humana e da solidariedade, aplicação direta da Constituição às relações privadas, proteção das pessoas vulneráveis, direitos da personalidade, relações familiares, contratos e responsabilidade civil."
2,"Possui graduação em Engenharia Mecânica pela Pontifícia Universidade Católica do Rio de 

In [198]:
# https://lars76.github.io/2018/05/08/portuguese-lemmatizers.html#2
!pip install stanza
import stanza
stanza.download('pt')

2021-10-28 21:52:53 INFO: Downloading default packages for language: pt (Portuguese)...
2021-10-28 21:52:54 INFO: File exists: /root/stanza_resources/pt/default.zip.
2021-10-28 21:52:56 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
stnz = stanza.Pipeline('pt', use_gpu=False)
documents_df['documents'] = documents_df['documents'].apply(lambda x: " ".join(str(word.lemma for sent in stnz(x).sentences for word in sent.words) for x in x.split()))

2021-10-28 22:35:43 INFO: Loading these models for language: pt (Portuguese):
| Processor | Package |
-----------------------
| tokenize  | bosque  |
| mwt       | bosque  |
| pos       | bosque  |
| lemma     | bosque  |
| depparse  | bosque  |

2021-10-28 22:35:43 INFO: Use device: cpu
2021-10-28 22:35:43 INFO: Loading: tokenize
2021-10-28 22:35:43 INFO: Loading: mwt
2021-10-28 22:35:43 INFO: Loading: pos
2021-10-28 22:35:45 INFO: Loading: lemma
2021-10-28 22:35:46 INFO: Loading: depparse


In [201]:
# removing special characters and stop words from the text
documents_df['documents']=documents_df['documents'].apply(lambda x: " ".join(normalize('NFD', x).encode('ascii', 'ignore').decode("utf-8") for x in x.split() ))
documents_df['documents'] = documents_df['documents'].apply(lambda x: " ".join(re.sub('[^a-zA-Z \n]', ' ', x).lower() for x in x.split()))
stop = stopwords.words('portuguese')
stop.extend(['puc-rio', 'puc', 'rio', 'janeiro', 'pontificia', 'universidade', 'catolica'])
documents_df['documents'] = documents_df['documents'].apply(lambda x: " ".join(Word(x).lemmatize() for x in x.split() if x not in stop))



In [202]:
documents_df

documents
0  id text tornar lemma tornar upos verb feat verbform inf head deprel root start char end char id text referencia lemma referencia upos noun feat gender fem number sing head deprel root start char end char id text nacional lemma nacional upos adj feat gender masc number sing head deprel root start char end char id text internacional lemma internacional upos adj feat gender masc number sing head deprel root start char end char id text meio lemma meio upos noun feat gender masc number sing head deprel root start char end char id text ambiente lemma ambiente upos noun feat gender masc number sing head deprel root start char end char id text contribuindo lemma contribuir upos verb feat verbform ger head deprel root start char end char id text atraves lemma atrave upos noun feat gender masc number sing head deprel root start char end char id text ciencia lemma ciencia upos noun feat gender fem number sing head deprel root start char end char id text educacao lemma educacao upos noun feat gender fem number sing head deprel root start char end char id text desenvolvimento lemma desenvolvimento upos noun feat gender masc number sing head deprel root start char end char id text sustentavel lemma sustentavel upos adj feat gender masc number sing head deprel root start char end char id text visando lemma visar upos verb feat verbform ger head deprel root start char end char id text estabelecer lemma estabelecer upos verb feat verbform inf head deprel root start char end char id text interacao lemma interacao upos noun feat gender masc number sing head deprel root start char end char id text meio lemma meio upos noun feat gender masc number sing head deprel root start char end char id text sociedade lemma sociedade upos noun feat gender fem number sing head deprel root start char end char id text natureza lemma natureza upos noun feat gender fem number sing head deprel root start char end char id text instituicao lemma instituicao upos noun feat gender fem number sing head deprel root start char end char id text direito lemma direito upos noun feat gender masc number sing head deprel root start char end char id text privado lemma privado upos adj feat gender masc number sing head deprel root start char end char id text fin lemma fin upos noun feat gender masc number sing head deprel root start char end char id text lucrativos lemma lucrativo upos noun feat gender masc number plur head deprel root start char end char id text prima lemma primo upos noun feat gender fem number sing head deprel root start char end char id text producao lemma producao upos noun feat gender fem number sing head deprel root start char end char id text transmissao lemma transmissao upos noun feat gender fem number sing head deprel root start char end char id text saber lemma saber upos verb feat verbform inf head deprel root start char end char id text baseando lemma basear upos verb feat verbform ger head deprel root start char end char id text respeito lemma respeito upos noun feat gender masc number sing head deprel root start char end char id text valores lemma valor upos noun feat gender masc number plur head deprel root start char end char id text humanos lemma humano upos noun feat gender masc number plur head deprel root start char end char id text etica lemma etica upos noun feat gender fem number sing head deprel root start char end char id text crista lemma crista upos noun feat gender fem number sing head deprel root start char end char id text visando lemma visar upos verb feat verbform ger head deprel root start char end char id text acima lemma acima upos adv head deprel root start char end char id text tudo lemma tudo upos pron feat gender masc number sing prontype ind head deprel root start char end char id text beneficio lemma beneficio upos noun feat gender masc number sing head deprel root start char end char id text sociedade lemma sociedade upos noun feat gender fem number sing head deprel root start char end char id text busca le

#### Tf-idf vectors

TfidfVectorizer: Convert a collection of raw documents to a matrix of TF-IDF features. Equivalent to CountVectorizer followed by TfidfTransformer.

In [150]:
tfidfvectoriser=TfidfVectorizer(max_features=64)
tfidfvectoriser.fit(documents_df['documents'])
tfidf_vectors=tfidfvectoriser.transform(documents_df['documents'])

In [151]:
tfidf_vectors.shape

(7, 64)

In [152]:
# Every vector is already normalised to have unit L2 norm
np.linalg.norm(tfidf_vectors.toarray()[0],ord=2)

1.0

In [153]:
tfidf_vectors=tfidf_vectors.toarray()
print (tfidf_vectors[0])

[0.16315106 0.34491371 0.         0.190872   0.         0.190872
 0.         0.190872   0.190872   0.         0.         0.
 0.         0.         0.095436   0.08157553 0.         0.
 0.08157553 0.         0.         0.16315106 0.         0.
 0.08157553 0.08157553 0.         0.         0.095436   0.
 0.         0.         0.095436   0.         0.         0.45988494
 0.         0.         0.45988494 0.         0.         0.190872
 0.         0.         0.         0.         0.         0.
 0.095436   0.08157553 0.         0.         0.08157553 0.
 0.         0.         0.095436   0.         0.34491371 0.095436
 0.095436   0.         0.         0.        ]


##### Every document has been converted into a 64 dimensional vector. As we set the max_features=64

### Pairwise similarity

##### Pairwise cosine similarity would just be the dot product of the vectors becasue tf-idf vectors from sklearn are already normalized and L2 norm of these vectors is 1. So denominator of cosine similiarity formula is 1 in this case.

In [154]:
pairwise_similarities=np.dot(tfidf_vectors,tfidf_vectors.T)
pairwise_differences=euclidean_distances(tfidf_vectors)

In [155]:
print (tfidf_vectors[0])
print (pairwise_similarities.shape)
print (pairwise_similarities[0][:])

[0.16315106 0.34491371 0.         0.190872   0.         0.190872
 0.         0.190872   0.190872   0.         0.         0.
 0.         0.         0.095436   0.08157553 0.         0.
 0.08157553 0.         0.         0.16315106 0.         0.
 0.08157553 0.08157553 0.         0.         0.095436   0.
 0.         0.         0.095436   0.         0.         0.45988494
 0.         0.         0.45988494 0.         0.         0.190872
 0.         0.         0.         0.         0.         0.
 0.095436   0.08157553 0.         0.         0.08157553 0.
 0.         0.         0.095436   0.         0.34491371 0.095436
 0.095436   0.         0.         0.        ]
(7, 7)
[1.         0.09154832 0.07658    0.06129466 0.12290645 0.11829918
 0.07460487]


##### similarity is highest, 1 at index 0 becasue they are the same documents

In [156]:
def most_similar(doc_id,similarity_matrix,matrix):
    print (f'Document: {documents_df.iloc[doc_id]["documents"]}')
    print ('\n')
    print (f'Similar Documents using {matrix}:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix:
        if ix==doc_id:
            continue
        print('\n')
        print (f'Document: {documents_df.iloc[ix]["documents"]}')
        print (f'{matrix} : {similarity_matrix[doc_id][ix]}')

In [157]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Document: tornar referencia nacional internacional meio ambiente contribuindo atraves ciencia educacao desenvolvimento sustentavel visando estabelecer interacao meio sociedade natureza instituicao direito privado fin lucrativos prima producao transmissao saber baseando respeito valores humanos etica crista visando acima tudo beneficio sociedade busca excelencia pesquisa ensino extensao formacao profissionais competentes habilitados pleno desempenho funcoes profissionais sao inseridos realidade brasileira formados colocar ciencia tecnica sempre servico homem colaborando atraves conhecimentos adquiridos construcao mundo melhor acordo exigencias justica amor cristao nucleo interdisciplinar meio ambiente nima fundacao ano atual reitor pe josafa carlos siqueira s j objetivo ser local discussoes interdisciplinares sobre questoes socioambientais alem aglutinar competencias internas nima tambem propoe estabelecer interacao meio cabe unidades complementares acordo estatuto ha doze ano nima vem 

In [158]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Document: tornar referencia nacional internacional meio ambiente contribuindo atraves ciencia educacao desenvolvimento sustentavel visando estabelecer interacao meio sociedade natureza instituicao direito privado fin lucrativos prima producao transmissao saber baseando respeito valores humanos etica crista visando acima tudo beneficio sociedade busca excelencia pesquisa ensino extensao formacao profissionais competentes habilitados pleno desempenho funcoes profissionais sao inseridos realidade brasileira formados colocar ciencia tecnica sempre servico homem colaborando atraves conhecimentos adquiridos construcao mundo melhor acordo exigencias justica amor cristao nucleo interdisciplinar meio ambiente nima fundacao ano atual reitor pe josafa carlos siqueira s j objetivo ser local discussoes interdisciplinares sobre questoes socioambientais alem aglutinar competencias internas nima tambem propoe estabelecer interacao meio cabe unidades complementares acordo estatuto ha doze ano nima vem 

#### word2vec embeddings

In [159]:
from keras.preprocessing.text import Tokenizer
import gensim
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [160]:
# tokenize and pad every document to make them of the same size
tokenizer=Tokenizer()
tokenizer.fit_on_texts(documents_df['documents'])
tokenized_documents=tokenizer.texts_to_sequences(documents_df['documents'])
tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=64,padding='post')
vocab_size=len(tokenizer.word_index)+1

print (tokenized_paded_documents[0])

[241 242 243  46 244 245 246 247 248 249 250 251  86  87 252 253 254 255
 256 257 258 259  88 260 261 262 263 264 265 266  22 267  85 268 269 270
  88 271 272 273 274  47 275 276 277 278 279 280 281 282 283 284  48 285
 286 287 288 289  89 290 291 292  90 293]


We can train our own embeddings if have enough data and computation available or we can use pre-trained embeddings. We will use a pre-trained embedding provided by Google.

In [161]:
# loading pre-trained embeddings, each word is represented as a 300 dimensional vector
import gensim
W2V_PATH="https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
model_w2v = gensim.models.KeyedVectors.load_word2vec_format(W2V_PATH, binary=True)

In [162]:
# creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index. 
embedding_matrix=np.zeros((vocab_size,300))
for word,i in tokenizer.word_index.items():
    if word in model_w2v:
        embedding_matrix[i]=model_w2v[word]

In [163]:
embedding_matrix[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [164]:
embedding_matrix[tokenizer.word_index['sustentavel']]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [165]:
# creating document-word embeddings
document_word_embeddings=np.zeros((len(tokenized_paded_documents),64,300))

for i in range(len(tokenized_paded_documents)):
    for j in range(len(tokenized_paded_documents[0])):
        document_word_embeddings[i][j]=embedding_matrix[tokenized_paded_documents[i][j]]

In [166]:
document_word_embeddings.shape

(7, 64, 300)

Now we have to represent every document as a single vector. We can either average or sum over every word vector and convert every 64X300 representation into a 300-dimensional representation. But averaging or summing over all the words would lose the semantic and contextual meaning of the documents. Different lengths of the documents would also have an adverse effect on such operations.

One better way of doing this could be taking a weighted average of word vectors using the tf-idf weights. This can handle the variable length problem to a certain extent but cannot keep the semantic and contextual meaning of words. After doing that we can use the pairwise distances to calculate similar documents as we did in the tf-idf model.

In [167]:
# tf-idf vectors do not keep the original sequence of words, converting them into actual word sequences from the documents

document_embeddings=np.zeros((len(tokenized_paded_documents),300))
words=tfidfvectoriser.get_feature_names()

for i in range(len(document_word_embeddings)):
    for j in range(len(words)):
        document_embeddings[i]+=embedding_matrix[tokenizer.word_index[words[j]]]*tfidf_vectors[i][j]
        
document_embeddings=document_embeddings/np.sum(tfidf_vectors,axis=1).reshape(-1,1)
    

In [168]:
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

In [169]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Document: tornar referencia nacional internacional meio ambiente contribuindo atraves ciencia educacao desenvolvimento sustentavel visando estabelecer interacao meio sociedade natureza instituicao direito privado fin lucrativos prima producao transmissao saber baseando respeito valores humanos etica crista visando acima tudo beneficio sociedade busca excelencia pesquisa ensino extensao formacao profissionais competentes habilitados pleno desempenho funcoes profissionais sao inseridos realidade brasileira formados colocar ciencia tecnica sempre servico homem colaborando atraves conhecimentos adquiridos construcao mundo melhor acordo exigencias justica amor cristao nucleo interdisciplinar meio ambiente nima fundacao ano atual reitor pe josafa carlos siqueira s j objetivo ser local discussoes interdisciplinares sobre questoes socioambientais alem aglutinar competencias internas nima tambem propoe estabelecer interacao meio cabe unidades complementares acordo estatuto ha doze ano nima vem 

In [170]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Document: tornar referencia nacional internacional meio ambiente contribuindo atraves ciencia educacao desenvolvimento sustentavel visando estabelecer interacao meio sociedade natureza instituicao direito privado fin lucrativos prima producao transmissao saber baseando respeito valores humanos etica crista visando acima tudo beneficio sociedade busca excelencia pesquisa ensino extensao formacao profissionais competentes habilitados pleno desempenho funcoes profissionais sao inseridos realidade brasileira formados colocar ciencia tecnica sempre servico homem colaborando atraves conhecimentos adquiridos construcao mundo melhor acordo exigencias justica amor cristao nucleo interdisciplinar meio ambiente nima fundacao ano atual reitor pe josafa carlos siqueira s j objetivo ser local discussoes interdisciplinares sobre questoes socioambientais alem aglutinar competencias internas nima tambem propoe estabelecer interacao meio cabe unidades complementares acordo estatuto ha doze ano nima vem 

#### Glove embeddings

Global Vectors for word Embedding (GloVe) is an unsupervised learning algorithm to produce vector representations of word. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space.

We will use the pre-trained Glove embeddings from Stanford. All the steps would remain same as word2vec embeddings it’s just that in this case we will use the Glove pre-trained model. We are using Glove embeddings of 100-dimensions because of the large size of the embeddings file. You can use higher dimensions also

In [171]:
# reading Glove word embeddings into a dictionary with "word" as key and values as word vectors

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

embeddings_index = dict()

with open('glove.6B.100d.txt', encoding='utf-8') as file:
    for line in file:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    

--2021-10-28 21:40:00--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-10-28 21:40:01--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-10-28 21:40:01--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

glove

In [172]:
# creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index. 

embedding_matrix=np.zeros((vocab_size,100))

for word,i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [173]:
# tf-idf vectors do not keep the original sequence of words, converting them into actual word sequences from the documents

document_embeddings=np.zeros((len(tokenized_paded_documents),100))
words=tfidfvectoriser.get_feature_names()

for i in range(documents_df.shape[0]):
    for j in range(len(words)):
        document_embeddings[i]+=embedding_matrix[tokenizer.word_index[words[j]]]*tfidf_vectors[i][j]
        
document_embeddings=document_embeddings/np.sum(tfidf_vectors,axis=1).reshape(-1,1)

In [174]:
document_embeddings.shape

(7, 100)

In [175]:
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

In [176]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Document: tornar referencia nacional internacional meio ambiente contribuindo atraves ciencia educacao desenvolvimento sustentavel visando estabelecer interacao meio sociedade natureza instituicao direito privado fin lucrativos prima producao transmissao saber baseando respeito valores humanos etica crista visando acima tudo beneficio sociedade busca excelencia pesquisa ensino extensao formacao profissionais competentes habilitados pleno desempenho funcoes profissionais sao inseridos realidade brasileira formados colocar ciencia tecnica sempre servico homem colaborando atraves conhecimentos adquiridos construcao mundo melhor acordo exigencias justica amor cristao nucleo interdisciplinar meio ambiente nima fundacao ano atual reitor pe josafa carlos siqueira s j objetivo ser local discussoes interdisciplinares sobre questoes socioambientais alem aglutinar competencias internas nima tambem propoe estabelecer interacao meio cabe unidades complementares acordo estatuto ha doze ano nima vem 

In [177]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Document: tornar referencia nacional internacional meio ambiente contribuindo atraves ciencia educacao desenvolvimento sustentavel visando estabelecer interacao meio sociedade natureza instituicao direito privado fin lucrativos prima producao transmissao saber baseando respeito valores humanos etica crista visando acima tudo beneficio sociedade busca excelencia pesquisa ensino extensao formacao profissionais competentes habilitados pleno desempenho funcoes profissionais sao inseridos realidade brasileira formados colocar ciencia tecnica sempre servico homem colaborando atraves conhecimentos adquiridos construcao mundo melhor acordo exigencias justica amor cristao nucleo interdisciplinar meio ambiente nima fundacao ano atual reitor pe josafa carlos siqueira s j objetivo ser local discussoes interdisciplinares sobre questoes socioambientais alem aglutinar competencias internas nima tambem propoe estabelecer interacao meio cabe unidades complementares acordo estatuto ha doze ano nima vem 

#### Doc2vec model

Doc2vec is an unsupervised learning algorithm to produce vector representations of sentence/paragraph/documents. This is an adaptation of word2vec. Doc2vec can represent an entire documents into a vector. So we don’t have to take average of word vectors to create document vector.


In [178]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [179]:
tagged_data = [TaggedDocument(words=word_tokenize(doc), tags=[i]) for i, doc in enumerate(documents_df['documents'])]

In [180]:
model_d2v = Doc2Vec(vector_size=100,alpha=0.025, min_count=1)
  
model_d2v.build_vocab(tagged_data)

for epoch in range(100):
    model_d2v.train(tagged_data,
                total_examples=model_d2v.corpus_count,
                epochs=model_d2v.epochs)

In [181]:
document_embeddings=np.zeros((documents_df.shape[0],100))

for i in range(len(document_embeddings)):
    document_embeddings[i]=model_d2v.docvecs[i]

In [182]:
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

In [183]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Document: tornar referencia nacional internacional meio ambiente contribuindo atraves ciencia educacao desenvolvimento sustentavel visando estabelecer interacao meio sociedade natureza instituicao direito privado fin lucrativos prima producao transmissao saber baseando respeito valores humanos etica crista visando acima tudo beneficio sociedade busca excelencia pesquisa ensino extensao formacao profissionais competentes habilitados pleno desempenho funcoes profissionais sao inseridos realidade brasileira formados colocar ciencia tecnica sempre servico homem colaborando atraves conhecimentos adquiridos construcao mundo melhor acordo exigencias justica amor cristao nucleo interdisciplinar meio ambiente nima fundacao ano atual reitor pe josafa carlos siqueira s j objetivo ser local discussoes interdisciplinares sobre questoes socioambientais alem aglutinar competencias internas nima tambem propoe estabelecer interacao meio cabe unidades complementares acordo estatuto ha doze ano nima vem 

In [184]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Document: tornar referencia nacional internacional meio ambiente contribuindo atraves ciencia educacao desenvolvimento sustentavel visando estabelecer interacao meio sociedade natureza instituicao direito privado fin lucrativos prima producao transmissao saber baseando respeito valores humanos etica crista visando acima tudo beneficio sociedade busca excelencia pesquisa ensino extensao formacao profissionais competentes habilitados pleno desempenho funcoes profissionais sao inseridos realidade brasileira formados colocar ciencia tecnica sempre servico homem colaborando atraves conhecimentos adquiridos construcao mundo melhor acordo exigencias justica amor cristao nucleo interdisciplinar meio ambiente nima fundacao ano atual reitor pe josafa carlos siqueira s j objetivo ser local discussoes interdisciplinares sobre questoes socioambientais alem aglutinar competencias internas nima tambem propoe estabelecer interacao meio cabe unidades complementares acordo estatuto ha doze ano nima vem 

#### BERT model

BERT- Bidirectional Encoder Representation from Transformers (BERT) is a state of the art technique for natural language processing pre-training developed by Google. BERT is trained on unlabelled text including Wikipedia and Book corpus. BERT uses transformer architecture, an attention model to learn embeddings for words.

BERT consists of two pre training steps Masked Language Modelling (MLM) and Next Sentence Prediction (NSP). In BERT training text is represented using three embeddings, Token Embeddings + Segment Embeddings + Position Embeddings.

In [185]:
# Install the library using pip
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

In [186]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [187]:
document_embeddings = sbert_model.encode(documents_df['documents'])

In [188]:
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

In [189]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Document: tornar referencia nacional internacional meio ambiente contribuindo atraves ciencia educacao desenvolvimento sustentavel visando estabelecer interacao meio sociedade natureza instituicao direito privado fin lucrativos prima producao transmissao saber baseando respeito valores humanos etica crista visando acima tudo beneficio sociedade busca excelencia pesquisa ensino extensao formacao profissionais competentes habilitados pleno desempenho funcoes profissionais sao inseridos realidade brasileira formados colocar ciencia tecnica sempre servico homem colaborando atraves conhecimentos adquiridos construcao mundo melhor acordo exigencias justica amor cristao nucleo interdisciplinar meio ambiente nima fundacao ano atual reitor pe josafa carlos siqueira s j objetivo ser local discussoes interdisciplinares sobre questoes socioambientais alem aglutinar competencias internas nima tambem propoe estabelecer interacao meio cabe unidades complementares acordo estatuto ha doze ano nima vem 

In [190]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Document: tornar referencia nacional internacional meio ambiente contribuindo atraves ciencia educacao desenvolvimento sustentavel visando estabelecer interacao meio sociedade natureza instituicao direito privado fin lucrativos prima producao transmissao saber baseando respeito valores humanos etica crista visando acima tudo beneficio sociedade busca excelencia pesquisa ensino extensao formacao profissionais competentes habilitados pleno desempenho funcoes profissionais sao inseridos realidade brasileira formados colocar ciencia tecnica sempre servico homem colaborando atraves conhecimentos adquiridos construcao mundo melhor acordo exigencias justica amor cristao nucleo interdisciplinar meio ambiente nima fundacao ano atual reitor pe josafa carlos siqueira s j objetivo ser local discussoes interdisciplinares sobre questoes socioambientais alem aglutinar competencias internas nima tambem propoe estabelecer interacao meio cabe unidades complementares acordo estatuto ha doze ano nima vem 

## Perform Semantic Search

In [192]:
import scipy
#@title Sematic Search Form

# code adapted from https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py

query = 'meio ambiente' #@param {type: 'string'}

queries = [query]
query_embeddings = sbert_model.encode(queries)

# Find the closest 5 documents of the corpus for each query sentence based on cosine similarity
number_top_matches = 5 #@param {type: "number"}

print("Semantic Search Results")

for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], document_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar documents in corpus:")

    for idx, distance in results[0:number_top_matches]:
        print(documents_df['documents'][idx].strip(), "(Cosine Score: %.4f)" % (1-distance))

Semantic Search Results




Query: meio ambiente

Top 5 most similar documents in corpus:
flavia almeida viveiros castro juiza direito professora convidada emerj escola magistratura estado esaj escola administracao judiciaria tribunal justica estado fundacao getulio vargas rj possui po doutorado direitos humanos instituto jus gentium faculdade direito coimbra doutora direito civil constitucional direito estado mestre direito mestre ciencia politica ciencia politica sociologia possui curso especializacao direito portugues centro estudos judiciarios portugal especializacao ciencia politica instituto metodista bennett graduada direito comunicacao social especialidade jornalismo juiza titular vara civel regional barra tijuca tendo exercido magistratura vara falencia fazenda publica criminal tribunal juri orfaos sucessoes familia ja assumiu funcao juiza eleitoral zona eleitoral ja exerceu mesma funcao marechal hermes bangu pavuna juiza corregedora auxiliar corregedora justica exerce docenci